In [1]:
load('quandles-alpha.sage')

In [29]:
# SageMath-ready PD analyzer
# Paste this into a Sage cell (sage kernel) and run.

from sage.all import *
import math

def unique_sorted(iterable):
    seen = []
    for x in iterable:
        if x not in seen:
            seen.append(x)
    return sorted(seen)

def place_arc_points_on_circle_sage(arcs, radius=1.0):
    """
    Place each arc label deterministically on a circle and return a dict label -> (x,y)
    Uses a small deterministic phase jitter per label to avoid perfect symmetry.
    """
    m = len(arcs)
    pts = {}
    for i, a in enumerate(arcs):
        theta = 2 * math.pi * i / max(1, m)
        # deterministic small jitter based on label
        j = ((int(a) * 97) % 1000 - 500) / 100000.0
        th = theta + j
        x = radius * math.cos(th)
        y = radius * math.sin(th)
        pts[a] = (float(x), float(y))
    return pts

def vector_sub(p, q):
    return (p[0]-q[0], p[1]-q[1])

def det2(u, v):
    return u[0]*v[1] - u[1]*v[0]

def normalize(v):
    norm = math.hypot(v[0], v[1])
    if norm == 0:
        return (0.0, 0.0)
    return (v[0]/norm, v[1]/norm)

def safe_tangent(p_from, p_to):
    v = vector_sub(p_to, p_from)
    if abs(v[0]) < 1e-12 and abs(v[1]) < 1e-12:
        # tiny deterministic fallback
        v = (1e-6, 0.0)
    return normalize(v)

def analyze_pd_sage(pd, ensure_non_degenerate=True):
    """
    Analyze PD in Sage. pd is a list of 4-tuples (a,b,c,d) where (a->b) is under strand,
    (c->d) is over strand. Returns a dictionary with arcs, pts, crossings and arc->crossing map.
    """
    # collect arcs
    arcs = []
    for tup in pd:
        arcs.extend([tup[0], tup[1], tup[2], tup[3]])
    arcs = unique_sorted(arcs)
    # place points
    pts = place_arc_points_on_circle_sage(arcs, radius=1.0)

    crossings = {}
    arc_to_crossings = {a: [] for a in arcs}

    for i, tup in enumerate(pd, start=1):
        a,b,c,d = tup
        p_a = pts[a]; p_b = pts[b]; p_c = pts[c]; p_d = pts[d]

        # compute tangents (under and over)
        # use direction from a->b for under, c->d for over (consistent orientation)
        t_under = safe_tangent(p_b, p_a)
        t_over  = safe_tangent(p_d, p_c)
        det = det2(t_under, t_over)
        # evaluate sign with tolerance
        tol = 1e-9
        if det > tol:
            sign = 1
        elif det < -tol:
            sign = -1
        else:
            sign = 0

        # if degenerate, attempt small deterministic perturbations
        if sign == 0 and ensure_non_degenerate:
            success = False
            for trial in range(6):
                # copy pts and perturb only involved labels slightly based on trial and label
                pts_trial = dict(pts)
                for label in (a,b,c,d):
                    x,y = pts_trial[label]
                    eps = (0.00025 + 0.00005*trial) * (((int(label)*31) % 7) - 3)
                    th = math.atan2(y,x) + eps
                    r = math.hypot(x,y)
                    pts_trial[label] = (r * math.cos(th), r * math.sin(th))
                p_a = pts_trial[a]; p_b = pts_trial[b]; p_c = pts_trial[c]; p_d = pts_trial[d]
                t_under = safe_tangent(p_b, p_a)
                t_over  = safe_tangent(p_d, p_c)
                det_trial = det2(t_under, t_over)
                if abs(det_trial) > tol:
                    det = det_trial
                    sign = 1 if det > 0 else -1
                    pts = pts_trial
                    success = True
                    break
            if not success:
                # fallback: determine sign from a deterministic rule based on tuple ordering:
                # This fallback is only used when geometry fails to disambiguate.
                sign = 1

        crossings[i] = {
            'pd_tuple': tup,
            'under_arc': (a,b),
            'over_arc': (c,d),
            'arcs_involved': sorted({a,b,c,d}),
            't_under': t_under,
            't_over': t_over,
            'det': float(det),
            'sign': int(sign),
            'sign_label': '+' if sign == 1 else ('-' if sign == -1 else '?')
        }
        for label in (a,b,c,d):
            arc_to_crossings.setdefault(label, []).append(i)

    return {
        'arcs': arcs,
        'pts': pts,
        'crossings': crossings,
        'arc_to_crossings': arc_to_crossings
    }


# ---------------- Example run on figure-eight PD ----------------
pd_fig8 = [(4,5,8,1), (1,2,5,6), (6,7,2,3), (3,4,7,8)]
kd = analyze_pd_sage(pd_fig8)

print("Arcs:", kd['arcs'])
print("Crossings summary:")
for ci, info in kd['crossings'].items():
    print(" %d: pd=%s, arcs=%s, sign=%s, det=%.6g" % (ci, str(info['pd_tuple']), str(info['arcs_involved']), info['sign_label'], info['det']))
print("Arc -> crossings:", kd['arc_to_crossings'])


Arcs: [1, 2, 3, 4, 5, 6, 7, 8]
Crossings summary:
 1: pd=(4, 5, 8, 1), arcs=[1, 4, 5, 8], sign=+, det=-2.22045e-16
 2: pd=(1, 2, 5, 6), arcs=[1, 2, 5, 6], sign=-, det=-0.00387999
 3: pd=(6, 7, 2, 3), arcs=[2, 3, 6, 7], sign=+, det=0.00387999
 4: pd=(3, 4, 7, 8), arcs=[3, 4, 7, 8], sign=-, det=-0.00387999
Arc -> crossings: {1: [1, 2], 2: [2, 3], 3: [3, 4], 4: [1, 4], 5: [1, 2], 6: [2, 3], 7: [3, 4], 8: [1, 4]}


In [37]:
# Sage cell: compute signs first, then merge arcs for naming
from sage.all import *
import math

# ---------------- geometry helpers ----------------
def vector_sub(p, q): return (p[0]-q[0], p[1]-q[1])
def det2(u, v): return u[0]*v[1] - u[1]*v[0]
def normalize(v):
    n = math.hypot(v[0], v[1])
    return (v[0]/n, v[1]/n) if n else (0.0, 0.0)
def safe_tangent(p_from, p_to):
    v = vector_sub(p_to, p_from)
    if abs(v[0]) < 1e-12 and abs(v[1]) < 1e-12:
        v = (1e-6, 0.0)
    return normalize(v)

# ---------------- place arc points (original, distinct arcs) ----------------
def place_arc_points(arcs, radius=1.0):
    """Place each original arc label at a distinct point on a circle (deterministic jitter)."""
    m = max(1, len(arcs))
    pts = {}
    for i, a in enumerate(sorted(arcs)):
        th = 2 * math.pi * i / m
        j = ((int(a) * 37) % 1000 - 500) / 100000.0
        th += j
        pts[a] = (radius * math.cos(th), radius * math.sin(th))
    return pts

# ---------------- compute signs from original PD ----------------
def compute_signs_from_pd(pd):
    """
    Input: pd list of 4-tuples (a,b,c,d) using standard PD convention:
           (a->b) is under strand, (c->d) is over strand.
    Returns dict crossing_index -> info with det, sign, sign_label and original pd tuple.
    """
    # collect original arcs
    arc_set = sorted({x for tup in pd for x in tup})
    pts = place_arc_points(arc_set)
    crossings = {}
    tol = 1e-9
    for i, (a,b,c,d) in enumerate(pd, start=1):
        p_a, p_b, p_c, p_d = pts[a], pts[b], pts[c], pts[d]
        # choose tangent directions consistent with traversal (a->b under, c->d over)
        t_under = safe_tangent(p_b, p_a)
        t_over  = safe_tangent(p_d, p_c)
        det = det2(t_under, t_over)
        if det > tol:
            sign = 1
        elif det < -tol:
            sign = -1
        else:
            # attempt small deterministic perturbation of involved points if degenerate
            sign = 0
            for trial in range(6):
                pts_trial = dict(pts)
                for label in (a,b,c,d):
                    x,y = pts_trial[label]
                    eps = (0.00025 + 0.00005*trial) * (((int(label)*31) % 7) - 3)
                    th = math.atan2(y,x) + eps
                    r = math.hypot(x,y)
                    pts_trial[label] = (r * math.cos(th), r * math.sin(th))
                t_under = safe_tangent(pts_trial[b], pts_trial[a])
                t_over  = safe_tangent(pts_trial[d], pts_trial[c])
                det_trial = det2(t_under, t_over)
                if abs(det_trial) > tol:
                    det = det_trial
                    sign = 1 if det > 0 else -1
                    pts = pts_trial
                    break
            # if still zero, default to +1 to avoid None
            if sign == 0:
                sign = 1
        crossings[i] = {
            'pd_tuple_original': (a,b,c,d),
            'under_arc_original': (a,b),
            'over_arc_original': (c,d),
            'det': float(det),
            'sign': int(sign),
            'sign_label': '+' if sign==1 else '-'
        }
    return crossings, arc_set

# ---------------- merge arcs by over-arc connectivity ----------------
def merge_arcs_from_pd(pd):
    """
    Merge arc labels according to over-arc connectivity:
      • for each over arc (c,d), merge c and d
      • merging is transitive
      • under arcs (a,b) remain distinct
    """
    arc_set = set()
    for a,b,c,d in pd:
        arc_set.update([a,b,c,d])
    parent = {x:x for x in arc_set}
    def find(x):
        while parent[x] != x:
            parent[x] = parent[parent[x]]
            x = parent[x]
        return x
    def union(x,y):
        rx, ry = find(x), find(y)
        if rx==ry: return
        if rx<ry: parent[ry]=rx
        else: parent[rx]=ry
    # merge using only over arcs
    for a,b,c,d in pd:
        union(c,d)

    groups={}
    for x in sorted(arc_set):
        r=find(x)
        groups.setdefault(r,[]).append(x)

    reps=sorted(groups.keys())
    rep_to_gid={rep:i+1 for i,rep in enumerate(reps)}
    gid_to_members={rep_to_gid[r]:groups[r] for r in reps}
    label_to_group={label:rep_to_gid[find(label)] for label in arc_set}

    pd_merged=[(label_to_group[a],label_to_group[b],
                 label_to_group[c],label_to_group[d]) for a,b,c,d in pd]
    return {
        'groups':gid_to_members,
        'label_to_group':label_to_group,
        'pd_merged':pd_merged,
        'reduced_arcs':sorted(gid_to_members.keys())
    }
# ---------------- main analyzer: signs first, then merge ----------------
def analyze_pd_sage(pd):
    # 1) compute crossing signs from original PD geometry
    crossings_raw, original_arcs = compute_signs_from_pd(pd)

    # 2) merge arcs (over-arc connectivity)
    merged = merge_arcs_from_pd(pd)
    pdm = merged['pd_merged']
    merged_arcs = merged['reduced_arcs']

    # 3) build arc->crossings mapping using merged labels
    arc_to_crossings = {g: [] for g in merged_arcs}
    crossings_final = {}
    # For each crossing index preserve sign computed earlier, but attach merged pd tuple
    for i, orig_info in crossings_raw.items():
        # original PD tuple:
        a,b,c,d = orig_info['pd_tuple_original']
        # merged tuple:
        a_m, b_m, c_m, d_m = merged['label_to_group'][a], merged['label_to_group'][b], merged['label_to_group'][c], merged['label_to_group'][d]
        # record final crossing info including preserved sign
        crossings_final[i] = {
            'pd_tuple_original': (a,b,c,d),
            'pd_tuple_merged': (a_m, b_m, c_m, d_m),
            'under_arc_merged': (a_m, b_m),
            'over_arc_merged': (c_m, d_m),
            'det_original': crossings_raw[i]['det'],
            'sign': crossings_raw[i]['sign'],
            'sign_label': crossings_raw[i]['sign_label']
        }
        # add to arc->crossing lists (use merged labels)
        for lab in (a_m, b_m, c_m, d_m):
            arc_to_crossings.setdefault(lab, []).append(i)

    # 4) build knot_info-like object (dict)
    knot_info = {
        'original_arcs': original_arcs,
        'merged': merged,
        'arcs': merged_arcs,
        'crossings': crossings_final,
        'arc_to_crossings': arc_to_crossings
    }
    return knot_info

# ---------------- Example: figure-eight PD ----------------
pd_fig8 = [(4,5,8,1), (1,2,5,6), (6,7,2,3), (3,4,7,8)]
kinfo = analyze_pd_sage(pd_fig8)

print("Original arcs (pre-merge):", kinfo['original_arcs'])
print("\nMerged groups (gID -> original arcs):")
for gid, members in kinfo['merged']['groups'].items():
    print(" g{}: {}".format(gid, members))
print("\nCrossings (merged tuples + signs):")
for ci,info in sorted(kinfo['crossings'].items()):
    print(" {}: pd_merged={}, sign={}, det(original)={:.6g}".format(ci, info['pd_tuple_merged'], info['sign_label'], info['det_original']))
print("\nArc -> crossings (merged labels):")
for a,xs in sorted(kinfo['arc_to_crossings'].items()):
    print(" arc {}: {}".format(a, xs))
print("\nTotal merged arcs:", len(kinfo['arcs']))


Original arcs (pre-merge): [1, 2, 3, 4, 5, 6, 7, 8]

Merged groups (gID -> original arcs):
 g1: [1, 7, 8]
 g2: [2, 3]
 g3: [4]
 g4: [5, 6]

Crossings (merged tuples + signs):
 1: pd_merged=(3, 4, 1, 1), sign=+, det(original)=0
 2: pd_merged=(1, 2, 4, 4), sign=-, det(original)=-0.00148
 3: pd_merged=(4, 1, 2, 2), sign=+, det(original)=0.00148
 4: pd_merged=(2, 3, 1, 1), sign=-, det(original)=-0.00148

Arc -> crossings (merged labels):
 arc 1: [1, 1, 2, 3, 4, 4]
 arc 2: [2, 3, 3, 4]
 arc 3: [1, 4]
 arc 4: [1, 2, 2, 3]

Total merged arcs: 4


In [40]:
import snappy
M = snappy.Link('3_1')
pd = M.PD_code()       # ← works for both knots and links
print(pd)

[(5, 2, 0, 3), (3, 0, 4, 1), (1, 4, 2, 5)]


In [38]:
M = snappy.Link('3_1')
pd = M.PD_code() 
kinfo = analyze_pd_sage(pd)

print("Original arcs (pre-merge):", kinfo['original_arcs'])
print("\nMerged groups (gID -> original arcs):")
for gid, members in kinfo['merged']['groups'].items():
    print(" g{}: {}".format(gid, members))
print("\nCrossings (merged tuples + signs):")
for ci,info in sorted(kinfo['crossings'].items()):
    print(" {}: pd_merged={}, sign={}, det(original)={:.6g}".format(ci, info['pd_tuple_merged'], info['sign_label'], info['det_original']))
print("\nArc -> crossings (merged labels):")
for a,xs in sorted(kinfo['arc_to_crossings'].items()):
    print(" arc {}: {}".format(a, xs))
print("\nTotal merged arcs:", len(kinfo['arcs']))


Original arcs (pre-merge): [0, 1, 2, 3, 4, 5]

Merged groups (gID -> original arcs):
 g1: [0, 3]
 g2: [1, 4]
 g3: [2, 5]

Crossings (merged tuples + signs):
 1: pd_merged=(3, 3, 1, 1), sign=+, det(original)=0.865655
 2: pd_merged=(1, 1, 2, 2), sign=+, det(original)=0.86621
 3: pd_merged=(2, 2, 3, 3), sign=+, det(original)=0.86621

Arc -> crossings (merged labels):
 arc 1: [1, 1, 2, 2]
 arc 2: [2, 2, 3, 3]
 arc 3: [1, 1, 3, 3]

Total merged arcs: 3


In [30]:
M = snappy.Link('3_1')
pd = M.PD_code() 
kd = analyze_pd_sage(pd)

print("Arcs:", kd['arcs'])
print("Crossings summary:")
for ci, info in kd['crossings'].items():
    print(" %d: pd=%s, arcs=%s, sign=%s, det=%.6g" % (ci, str(info['pd_tuple']), str(info['arcs_involved']), info['sign_label'], info['det']))
print("Arc -> crossings:", kd['arc_to_crossings'])


Arcs: [0, 1, 2, 3, 4, 5]
Crossings summary:
 1: pd=(5, 2, 0, 3), arcs=[0, 2, 3, 5], sign=+, det=0.865054
 2: pd=(3, 0, 4, 1), arcs=[0, 1, 3, 4], sign=+, det=0.86651
 3: pd=(1, 4, 2, 5), arcs=[1, 2, 4, 5], sign=+, det=0.86651
Arc -> crossings: {0: [1, 2], 1: [2, 3], 2: [1, 3], 3: [1, 2], 4: [2, 3], 5: [1, 3]}


In [39]:
# Debuggable analyze_pd_sage for Sage (signs from original PD, then merge)
import math
from collections import defaultdict

# ---------------- geometry helpers ----------------
def vector_sub(p, q): return (p[0]-q[0], p[1]-q[1])
def det2(u, v): return u[0]*v[1] - u[1]*v[0]
def normalize(v):
    n = math.hypot(v[0], v[1])
    return (v[0]/n, v[1]/n) if n else (0.0, 0.0)
def safe_tangent(p_from, p_to):
    v = vector_sub(p_to, p_from)
    if abs(v[0]) < 1e-12 and abs(v[1]) < 1e-12:
        v = (1e-6, 0.0)
    return normalize(v)

# ---------------- place arc points ----------------
def place_arc_points(arcs, radius=1.0):
    """Place each original arc label at a distinct point on a circle (deterministic jitter)."""
    arcs_sorted = list(sorted(arcs))
    m = max(1, len(arcs_sorted))
    pts = {}
    for i, a in enumerate(arcs_sorted):
        th = 2 * math.pi * i / m
        j = ((int(a) * 37) % 1000 - 500) / 100000.0
        th += j
        pts[a] = (radius * math.cos(th), radius * math.sin(th))
    return pts

# ---------------- compute signs from original PD ----------------
def compute_signs_from_pd(pd, debug=False):
    """
    Input: pd list of 4-tuples (a,b,c,d) using standard PD convention
           (under: a->b, over: c->d).
    Returns (crossings, arc_set, occurrences) where crossings is dict(index->info).
    """
    # Make sure pd is a list of 4-tuples
    pd = [tuple(t) for t in pd]
    # collect original arcs and occurrences
    occurrences = defaultdict(list)   # arc_label -> list of (crossing_index, role, position_in_tuple)
    for i, tup in enumerate(pd, start=1):
        if len(tup) != 4:
            raise ValueError(f"PD entry {i} is not length 4: {tup}")
        a,b,c,d = tup
        occurrences[a].append((i, 'under_enter', 0))
        occurrences[b].append((i, 'under_exit', 1))
        occurrences[c].append((i, 'over_enter', 2))
        occurrences[d].append((i, 'over_exit', 3))
    arc_set = sorted(occurrences.keys())

    if debug:
        print("Raw PD (as provided):")
        for i,tup in enumerate(pd, start=1):
            print(f" {i}: {tup}")
        print("\nArc occurrences (arc_label -> occurrences):")
        for arc in arc_set:
            print(f"  {arc}: {occurrences[arc]}")

    # Basic sanity check: each arc should appear exactly twice (for a single component)
    bad = [arc for arc, occ in occurrences.items() if len(occ) != 2]
    if bad and debug:
        print("\nWarning: some arcs do not occur exactly twice (this may indicate a non-standard PD or multi-component link):", bad)

    # place points using original arc labels
    pts = place_arc_points(arc_set)
    crossings = {}
    tol = 1e-9
    for i, (a,b,c,d) in enumerate(pd, start=1):
        p_a, p_b, p_c, p_d = pts[a], pts[b], pts[c], pts[d]
        t_under = safe_tangent(p_b, p_a)
        t_over = safe_tangent(p_d, p_c)
        det = det2(t_under, t_over)
        sign = None
        if det > tol:
            sign = 1
        elif det < -tol:
            sign = -1
        else:
            # small deterministic perturb attempt
            sign = 0
            for trial in range(6):
                pts_trial = dict(pts)
                for label in (a,b,c,d):
                    x,y = pts_trial[label]
                    eps = (0.00025 + 0.00005*trial) * (((int(label)*31) % 7) - 3)
                    th = math.atan2(y,x) + eps
                    r = math.hypot(x,y)
                    pts_trial[label] = (r * math.cos(th), r * math.sin(th))
                t_under = safe_tangent(pts_trial[b], pts_trial[a])
                t_over = safe_tangent(pts_trial[d], pts_trial[c])
                det_trial = det2(t_under, t_over)
                if abs(det_trial) > tol:
                    det = det_trial
                    sign = 1 if det > 0 else -1
                    pts = pts_trial
                    break
            if sign == 0:
                # final fallback
                sign = 1
        crossings[i] = {
            'pd_tuple_original': (a,b,c,d),
            'under_original': (a,b),
            'over_original': (c,d),
            'det_original': float(det),
            'sign': int(sign),
            'sign_label': '+' if sign==1 else '-'
        }
    return crossings, arc_set, occurrences

# ---------------- merge arcs by over-arc connectivity ----------------
def merge_arcs_from_pd(pd, debug=False):
    arc_set = set()
    for a,b,c,d in pd:
        arc_set.update([a,b,c,d])
    parent = {x:x for x in arc_set}
    def find(x):
        while parent[x] != x:
            parent[x] = parent[parent[x]]
            x = parent[x]
        return x
    def union(x,y):
        rx, ry = find(x), find(y)
        if rx==ry:
            return
        # deterministically choose smaller rep
        if rx < ry: parent[ry] = rx
        else: parent[rx] = ry
    # union over arcs only
    for a,b,c,d in pd:
        union(c,d)
    groups = {}
    for x in sorted(arc_set):
        r = find(x)
        groups.setdefault(r, []).append(x)
    # produce stable gid assignment in sorted order of representative
    reps = sorted(groups.keys())
    rep_to_gid = {rep: (i+1) for i,rep in enumerate(reps)}
    gid_to_members = {rep_to_gid[r]: groups[r] for r in reps}
    label_to_group = {label: rep_to_gid[find(label)] for label in arc_set}
    pd_merged = [(label_to_group[a], label_to_group[b], label_to_group[c], label_to_group[d]) for a,b,c,d in pd]
    if debug:
        print("\nMerge map (original -> group):")
        for lab in sorted(label_to_group):
            print(" ", lab, "-> g", label_to_group[lab])
        print("\nMerged PD (tuples):")
        for i,t in enumerate(pd_merged, start=1):
            print(" ", i, t)
    return {'groups': gid_to_members, 'label_to_group': label_to_group, 'pd_merged': pd_merged, 'reduced_arcs': sorted(gid_to_members.keys())}

# ---------------- high-level analyzer (signs first, merge second) ----------------
def analyze_pd_sage_debug(pd, debug=True):
    """
    Computes signs on original PD, then merges arcs for naming.
    Returns kinfo dict and prints debug info if debug=True.
    """
    if debug:
        print("=== Running analyze_pd_sage_debug ===")
    crossings_raw, original_arcs, occurrences = compute_signs_from_pd(pd, debug=debug)
    if debug:
        print("\nCrossings (original, computed signs):")
        for i,info in crossings_raw.items():
            print(f" {i}: orig_pd={info['pd_tuple_original']}, sign={info['sign_label']}, det={info['det_original']:.6g}")

    merged = merge_arcs_from_pd(pd, debug=debug)
    pd_merged = merged['pd_merged']
    merged_arcs = merged['reduced_arcs']

    # Build final crossing records attaching merged tuple but keep original sign
    crossings_final = {}
    arc_to_crossings = {g:[] for g in merged_arcs}
    for i, info in crossings_raw.items():
        a,b,c,d = info['pd_tuple_original']
        a_m, b_m, c_m, d_m = merged['label_to_group'][a], merged['label_to_group'][b], merged['label_to_group'][c], merged['label_to_group'][d]
        crossings_final[i] = {
            'pd_tuple_original': (a,b,c,d),
            'pd_tuple_merged': (a_m,b_m,c_m,d_m),
            'det_original': info['det_original'],
            'sign': info['sign'],
            'sign_label': info['sign_label']
        }
        for lab in (a_m,b_m,c_m,d_m):
            arc_to_crossings.setdefault(lab, []).append(i)

    kinfo = {
        'original_arcs': original_arcs,
        'merged': merged,
        'arcs': merged_arcs,
        'crossings': crossings_final,
        'arc_to_crossings': arc_to_crossings,
        'occurrences': occurrences
    }

    if debug:
        print("\nFinal summary (merged tuples + signs):")
        for ci,inf in sorted(kinfo['crossings'].items()):
            print(f" {ci}: pd_merged={inf['pd_tuple_merged']}, sign={inf['sign_label']}, det(original)={inf['det_original']:.6g}")
        print("\nArc -> crossings (merged labels):")
        for a,xs in sorted(kinfo['arc_to_crossings'].items()):
            print(" arc", a, ":", xs)
        print("\nTotal merged arcs:", len(kinfo['arcs']))
    return kinfo

# ---------------- Example you can replace pd_example with your PD ----------------
# Example PD for a trefoil-like input (you can paste your PD here)
pd_example = [(5,2,0,3), (3,0,4,1), (1,4,2,5)]
kinfo = analyze_pd_sage_debug(pd_example, debug=True)


=== Running analyze_pd_sage_debug ===
Raw PD (as provided):
 1: (5, 2, 0, 3)
 2: (3, 0, 4, 1)
 3: (1, 4, 2, 5)

Arc occurrences (arc_label -> occurrences):
  0: [(1, 'over_enter', 2), (2, 'under_exit', 1)]
  1: [(2, 'over_exit', 3), (3, 'under_enter', 0)]
  2: [(1, 'under_exit', 1), (3, 'over_enter', 2)]
  3: [(1, 'over_exit', 3), (2, 'under_enter', 0)]
  4: [(2, 'over_enter', 2), (3, 'under_exit', 1)]
  5: [(1, 'under_enter', 0), (3, 'over_exit', 3)]

Crossings (original, computed signs):
 1: orig_pd=(5, 2, 0, 3), sign=+, det=0.865655
 2: orig_pd=(3, 0, 4, 1), sign=+, det=0.86621
 3: orig_pd=(1, 4, 2, 5), sign=+, det=0.86621

Merge map (original -> group):
  0 -> g 1
  1 -> g 2
  2 -> g 3
  3 -> g 1
  4 -> g 2
  5 -> g 3

Merged PD (tuples):
  1 (3, 3, 1, 1)
  2 (1, 1, 2, 2)
  3 (2, 2, 3, 3)

Final summary (merged tuples + signs):
 1: pd_merged=(3, 3, 1, 1), sign=+, det(original)=0.865655
 2: pd_merged=(1, 1, 2, 2), sign=+, det(original)=0.86621
 3: pd_merged=(2, 2, 3, 3), sign=+, det

In [43]:
# ===========================
# Wirtinger relations -> group + substitution reducer
# ===========================
from sage.all import FreeGroup
from typing import List, Dict, Tuple
import re

# ---------- helper: convert merged arcs to generator names ----------
def mk_gen_names(merged_arcs: List[int]) -> Dict[int,str]:
    # map merged arc id -> generator symbol 'g1','g2',...
    mapping = {}
    for i, a in enumerate(sorted(merged_arcs), start=1):
        mapping[a] = f"g{i}"
    return mapping

# ---------- create group presentation in Sage ----------
def wirtinger_group_from_knotinfo(knot_info: Dict) -> Tuple:
    """
    Input: knot_info returned from analyze_pd_sage (with merged arcs)
    Returns: (F, gens_map, rels_words, G) where
      - F is the FreeGroup
      - gens_map maps merged-arc-id -> generator (Sage group generator object)
      - rels_words is a list of Sage words (relations)
      - G is the quotient group F / rels (presentation)
    """
    merged = knot_info['merged']
    merged_arcs = knot_info['arcs']  # list of merged arc ids
    gens_map_names = mk_gen_names(merged_arcs)  # e.g. {1:'g1', ...}

    # Build FreeGroup with those names
    names = [gens_map_names[a] for a in sorted(merged_arcs)]
    F = FreeGroup(names)
    gens = F.gens()
    # map names -> generator objects
    name_to_gen = {str(g): g for g in gens}   # keys 'g1','g2',...
    # map merged arc id -> generator object
    arcid_to_gen = {a: name_to_gen[gens_map_names[a]] for a in merged_arcs}

    # build relations according to signs
    rels = []
    for ci, info in sorted(knot_info['crossings'].items()):
        # merged pd tuple
        a_m, b_m, c_m, d_m = info['pd_tuple_merged']
        sign = info['sign']
        g_a = arcid_to_gen[a_m]
        g_b = arcid_to_gen[b_m]
        g_c = arcid_to_gen[c_m]
        # positive crossing: g_b = g_c^-1 * g_a * g_c  -> relation r = g_c^-1*g_a*g_c * g_b^-1
        if sign == 1:
            rel_word = g_c**-1 * g_a * g_c * g_b**-1
        else:
            # negative crossing: g_a = g_c^-1 * g_b * g_c  -> r = g_c^-1*g_b*g_c * g_a^-1
            rel_word = g_c**-1 * g_b * g_c * g_a**-1
        rels.append(rel_word)

    # quotient
    G = F.quotient(rels)
    return F, arcid_to_gen, rels, G

# ---------- small symbolic substitution reducer ----------
# We'll represent group words as lists of signed tokens like ['g1', 'g2^-1', ...]
def word_to_tokens(word_str: str) -> List[str]:
    """Convert a string like 'g1^-1*g2*g3^-1' -> ['g1^-1','g2','g3^-1']"""
    # normalize separators
    tokens = [t for t in re.split(r'\s*\*\s*', word_str) if t]
    return tokens

def invert_token(tok: str) -> str:
    if tok.endswith('^-1'):
        return tok[:-3]
    else:
        return tok + '^-1'

def multiply_tokens(a: List[str], b: List[str]) -> List[str]:
    # concatenate and reduce adjacent inverse pairs
    res = a.copy()
    for t in b:
        if res and invert_token(t) == res[-1]:
            res.pop()
        else:
            res.append(t)
    return res

def tokens_to_str(tokens: List[str]) -> str:
    return '*'.join(tokens) if tokens else '1'

def tokens_from_sage_word(sage_word) -> List[str]:
    # sage_word.str() gives something like 'g1^-1*g2' — parse that
    s = str(sage_word)
    if s == '1':
        return []
    return word_to_tokens(s.replace('**-1','^-1').replace('**1',''))

def substitute_once(eq_map: Dict[str,List[str]], tokens: List[str]) -> Tuple[List[str], bool]:
    """
    If any token in `tokens` equals a key in eq_map, replace it by eq_map[token] tokens.
    Return (new_tokens, changed)
    Note: eq_map keys should be bare tokens like 'g1' or 'g1^-1'.
    """
    changed = False
    res = []
    for t in tokens:
        # check direct match
        if t in eq_map:
            changed = True
            sub = eq_map[t]  # list of tokens to substitute in
            # multiply res by sub (with cancellation)
            res = multiply_tokens(res, sub)
        else:
            # also handle inverse substitution if inverse key exists
            if t.endswith('^-1'):
                base = t[:-3]
                if base in eq_map:
                    changed = True
                    # substitute base then invert the substituted word
                    sub = eq_map[base]
                    # invert sub: reverse order and invert each token
                    sub_inv = [invert_token(tok) for tok in reversed(sub)]
                    res = multiply_tokens(res, sub_inv)
                    continue
            # otherwise keep token
            res = multiply_tokens(res, [t])
    return res, changed

def reduce_relations_by_substitution(knot_info: Dict) -> Dict:
    """
    Attempt to reduce generators by substitution.
    Returns a dict with:
      - 'subs' : substitution mapping generator token -> token list
      - 'reduced_relations' : list of remaining relations (as token lists)
      - 'generators' : original generator names
    """
    # build initial relations as tokens
    merged_arcs = sorted(knot_info['arcs'])
    gen_names = [f"g{idx}" for idx in range(1, len(merged_arcs)+1)]
    # map merged arc ids (sorted) to generator names g1..gk
    arcid_to_gname = {a: f"g{i}" for i,a in enumerate(merged_arcs, start=1)}
    # inverse map if needed
    # Build raw relations of the form lhs == rhs as tokens
    eqs = []  # list of (lhs_token, rhs_tokens)
    for ci, info in sorted(knot_info['crossings'].items()):
        a_m, b_m, c_m, d_m = info['pd_tuple_merged']
        sign = info['sign']
        ga = arcid_to_gname[a_m]; gb = arcid_to_gname[b_m]; gc = arcid_to_gname[c_m]
        if sign == 1:
            # gb = gc^-1 * ga * gc
            rhs = [gc + '^-1', ga, gc]
            lhs = gb
        else:
            # ga = gc^-1 * gb * gc
            rhs = [gc + '^-1', gb, gc]
            lhs = ga
        eqs.append((lhs, rhs))

    # eq_map: token -> tokenlist for substitution (we keep substitutions like 'g2' -> ['g1','g3^-1'])
    eq_map = {}
    changed = True
    iteration = 0
    # Greedy substitution: if a relation has form single-lhs = rhs where rhs does not contain lhs,
    # record it in eq_map and substitute into all other relations.
    while changed and iteration < 100:
        changed = False
        iteration += 1
        # find a substitution candidate
        candidate = None
        for lhs, rhs in eqs:
            # if rhs is not trivial and lhs does not appear inside rhs (no self recursion), choose it
            if lhs not in rhs:
                # prefer RHS with fewer tokens (simpler substitution) — this is heuristic
                candidate = (lhs, rhs)
                break
        if candidate:
            lhs, rhs = candidate
            # add to eq_map but first fully substitute existing eq_map into rhs
            # (expand rhs tokens using existing eq_map recursively)
            tokens = rhs
            # apply existing substitutions until fixed
            subchanged = True
            while subchanged:
                subchanged = False
                new_tokens = []
                for t in tokens:
                    if t in eq_map:
                        subchanged = True
                        new_tokens = multiply_tokens(new_tokens, eq_map[t])
                    elif t.endswith('^-1') and t[:-3] in eq_map:
                        subchanged = True
                        sub = eq_map[t[:-3]]
                        sub_inv = [invert_token(tok) for tok in reversed(sub)]
                        new_tokens = multiply_tokens(new_tokens, sub_inv)
                    else:
                        new_tokens = multiply_tokens(new_tokens, [t])
                tokens = new_tokens
            eq_map[lhs] = tokens
            # remove any equations where lhs is on left (they're now recorded)
            eqs = [(L,R) for (L,R) in eqs if L != lhs]
            # substitute this new mapping into all rhs of remaining eqs
            new_eqs = []
            for L, R in eqs:
                R_tokens = []
                for t in R:
                    if t in eq_map:
                        R_tokens = multiply_tokens(R_tokens, eq_map[t])
                    elif t.endswith('^-1') and t[:-3] in eq_map:
                        sub = eq_map[t[:-3]]
                        sub_inv = [invert_token(tok) for tok in reversed(sub)]
                        R_tokens = multiply_tokens(R_tokens, sub_inv)
                    else:
                        R_tokens = multiply_tokens(R_tokens, [t])
                # if substitution created L inside RHS (self-recursive), keep equation as is (no mapping)
                new_eqs.append((L, R_tokens))
            eqs = new_eqs
            changed = True
        else:
            break

    # After substitution loop, build remaining relation strings
    reduced_relations = [(L, R) for L, R in eqs]
    return {
        'gens': gen_names,
        'arcid_to_gname': {a: f"g{i}" for i,a in enumerate(sorted(knot_info['arcs']), start=1)},
        'subs': eq_map,
        'reduced_relations': reduced_relations
    }

# ----------------- Useful wrapper that does both Sage group and substitution -----------------
def analyze_wirtinger_and_reduce(knot_info):
    F, arcid_to_gen, rels, G = wirtinger_group_from_knotinfo(knot_info)
    print("Sage free group gens:", [str(g) for g in F.gens()])
    print("Number of relations:", len(rels))
    print("Presentation (raw relations):")
    for r in rels:
        print(" ", r)
    # Try substitutions
    subres = reduce_relations_by_substitution(knot_info)
    print("\nSubstitution map (generator -> token list):")
    for k, v in subres['subs'].items():
        print(" ", k, "->", tokens_to_str(v))
    print("\nRemaining relations after greedy substitution:")
    for L, R in subres['reduced_relations']:
        print(" ", L, "=", tokens_to_str(R))
    return F, G, subres

# ===========================
# Example usage (use your knot_info from analyze_pd_sage)
# ===========================
# If you already have kinfo from analyze_pd_sage:
# F, G, subres = analyze_wirtinger_and_reduce(kinfo)

# If you don't, you can create one quickly for the trefoil example:
# Example format for knot_info expected by the function:
#   knot_info = {
#       'arcs': [1,2,3],
#       'merged': {...},
#       'crossings': {
#            1: {'pd_tuple_merged': (3,3,1,1), 'sign': 1, ...},
#            2: { ... },
#            ...
#       }
#   }
#
# Replace the `kinfo_example` below with your actual kinfo (or call analyze_pd_sage above).
#
# Example (trefoil-like) to test the routine:
kinfo_example = {
    'arcs': [1,2,3],
    'merged': {'groups': {1:[0,3], 2:[1,4], 3:[2,5]}, 'label_to_group': {}},
    'crossings': {
        1: {'pd_tuple_merged': (3,3,1,1), 'sign': 1},
        2: {'pd_tuple_merged': (1,1,2,2), 'sign': 1},
        3: {'pd_tuple_merged': (2,2,3,3), 'sign': 1}
    }
}
# Uncomment to test:
# F, G, subres = analyze_wirtinger_and_reduce(kinfo_example)


IndentationError: unindent does not match any outer indentation level (<string>, line 248)